In [1]:
import sys
import os
import sys
import os
# Add the parent directory of TRVQA to sys.path
sys.path.insert(0, os.path.abspath(os.path.join(os.getcwd(), '../..')))
from TRVQA.measure.enums import MeasureMethod
from TRVQA.optimization.gradients.vanilla_parameter_shift import vanilla_parameter_shift
from TRVQA.optimization.optimization import minimize, minimize_custom
from TRVQA.utils.maxcut import gengraph, create_hamiltonian, make_hamiltonian
import torch
from torch.autograd import Function
from qiskit.circuit.library import QAOAAnsatz
from qiskit.quantum_info import SparsePauliOp, Statevector
from qiskit import QuantumCircuit

import numpy as np
import torch
from TRVQA.circuit import Circuit

def compute_maxcut_value(bitstring, graph):
    """Computes the MaxCut value for a given bitstring solution."""
    cut_value = sum(w for (u,v), w in graph if bitstring[u] != bitstring[v])
    return cut_value

# for s in best_value:
#     print(compute_maxcut_value(s, g))

from pyparsing import Optional
import torch
import numpy as np
import matplotlib.pyplot as plt
from qiskit import QuantumCircuit
from qiskit.quantum_info import Statevector, Operator, SparsePauliOp
from TRVQA.optimization.optimization import minimize, minimize_custom
# TRVQA imports

from TRVQA.circuit import Circuit
from TRVQA.hamiltonian.hamiltonian import Hamiltonian
from TRVQA.measure.enums import MeasureMethod
torch.cuda.empty_cache()
import numpy as np
import torch
from qiskit import QuantumCircuit
from qiskit.quantum_info import Statevector, Operator, SparsePauliOp
import torch

from TRVQA.measure.enums import MeasureMethod
from TRVQA.optimization.optimization import minimize
from TRVQA.utils.maxcut import gengraph, create_hamiltonian,make_hamiltonian
from TRVQA.hamiltonian.hamiltonian import Hamiltonian
import numpy as np
import torch
from TRVQA.circuit import Circuit
import time


In [2]:
import random
import numpy as np
from itertools import product

# --- Helper: deterministic seeding for reproducibility ---
def _seed_everywhere(seed: int):
    random.seed(seed)
    try:
        np.random.seed(seed)
    except Exception:
        pass
    try:
        import torch
        torch.manual_seed(seed)
        if torch.cuda.is_available():
            torch.cuda.manual_seed_all(seed)
    except Exception:
        pass

# --- Robust graph generation with retries & deterministic seeds ---
def create_graphs_per_n(
    Ns,
    per_n: int = 10,
    max_retries: int = 100,
    base_seed: int = 12345,
    gengraph_fn=None,
):
    """
    Returns a list of dicts:
      {'N': int, 'graph_id': int, 'seed': int, 'G': networkx.Graph}
    """
    if gengraph_fn is None:
        # Fall back to a global gengraph if it's already defined in your notebook
        try:
            gengraph_fn = globals()['gengraph']
        except KeyError:
            raise RuntimeError("Provide gengraph_fn or define a global `gengraph(N, seed=None)`.")

    runs = []
    for N in Ns:
        for gid in range(per_n):
            # Stable seed per (N, graph_id)
            base = (base_seed * 1_000_003) ^ (N * 9_73_733) ^ (gid * 31_4159)
            for attempt in range(max_retries):
                # Jitter the seed per attempt for robustness but remain deterministic
                seed = (base + attempt) & 0x7FFFFFFF
                _seed_everywhere(seed)
                try:
                    # Prefer gengraph_fn(N, seed=seed) if supported; else call gengraph_fn(N)
                    try:
                        G = gengraph_fn(N, seed=seed)
                    except TypeError:
                        G = gengraph_fn(N)
                    runs.append({'N': N, 'graph_id': gid, 'seed': seed, 'G': G})
                    break
                except Exception as e:
                    if attempt == max_retries - 1:
                        print(f"[FAIL] N={N} gid={gid}: {e}")
                    # else retry with next seed
    return runs

# --- Build the experimental grid over depths (same depth set for all N) ---
def build_experiment_grid(graph_runs, depths):
    """
    graph_runs: output of create_graphs_per_n(...)
    depths: iterable of circuit depths (same for all N)
    Returns list of dicts:
      {'N', 'graph_id', 'seed', 'depth', 'G'}
    """
    grid = []
    for r in graph_runs:
        for d in depths:
            grid.append({**r, 'depth': d})
    return grid

# ------------------ Example usage ------------------
N_list = [150, 500, 1000]  # your Ns
per_n = 10                                     # 10 graphs per N
depths = [1]                   # SAME depth set used for every N

graph_runs = create_graphs_per_n(
    Ns=N_list,
    per_n=per_n,
    max_retries=100,
    base_seed=42,          # change if you want a different reproducible suite
    gengraph_fn=gengraph,  # or omit if gengraph is global
)

exp_grid = build_experiment_grid(graph_runs, depths)

print(f"Total graphs: {len(graph_runs)}  (expected {len(N_list)*per_n})")
print(f"Total (graph, depth) pairs: {len(exp_grid)}  (expected {len(graph_runs)*len(depths)})")
# Optional: quick peek at the first few items
for row in exp_grid:
    print(f"N={row['N']:<3} gid={row['graph_id']:<2} depth={row['depth']:<2} seed={row['seed']}")


Total graphs: 30  (expected 30)
Total (graph, depth) pairs: 30  (expected 30)
N=150 gid=0  depth=1  seed=171207760
N=150 gid=1  depth=1  seed=170960767
N=150 gid=2  depth=1  seed=171833870
N=150 gid=3  depth=1  seed=171576797
N=150 gid=4  depth=1  seed=170344684
N=150 gid=5  depth=1  seed=170105788
N=150 gid=6  depth=1  seed=170438474
N=150 gid=7  depth=1  seed=169206297
N=150 gid=8  depth=1  seed=168965416
N=150 gid=9  depth=1  seed=169822457
N=500 gid=0  depth=1  seed=528752828
N=500 gid=1  depth=1  seed=528542613
N=500 gid=2  depth=1  seed=529381092
N=500 gid=3  depth=1  seed=529154359
N=500 gid=4  depth=1  seed=529992710
N=500 gid=5  depth=1  seed=529782609
N=500 gid=6  depth=1  seed=530113441
N=500 gid=7  depth=1  seed=530951924
N=500 gid=8  depth=1  seed=530741698
N=500 gid=9  depth=1  seed=531563550
N=1000 gid=0  depth=1  seed=948511350
N=1000 gid=1  depth=1  seed=948824409
N=1000 gid=2  depth=1  seed=947958826
N=1000 gid=3  depth=1  seed=948388860
N=1000 gid=4  depth=1  seed=94

In [3]:
print(exp_grid[0])

{'N': 150, 'graph_id': 0, 'seed': 171207760, 'G': [((0, 41), 1), ((0, 73), 6), ((0, 7), 1), ((1, 57), 10), ((1, 42), 6), ((1, 86), 4), ((2, 29), 3), ((2, 129), 10), ((2, 10), 6), ((3, 4), 7), ((3, 84), 2), ((3, 128), 1), ((4, 121), 5), ((4, 89), 3), ((5, 119), 3), ((5, 137), 5), ((5, 18), 5), ((6, 103), 7), ((6, 87), 8), ((6, 70), 5), ((7, 112), 4), ((7, 54), 7), ((8, 84), 9), ((8, 83), 8), ((8, 21), 2), ((9, 63), 4), ((9, 110), 6), ((9, 35), 2), ((10, 55), 10), ((10, 78), 10), ((11, 65), 9), ((11, 24), 1), ((11, 46), 7), ((12, 91), 5), ((12, 115), 1), ((12, 80), 1), ((13, 96), 6), ((13, 111), 6), ((13, 14), 8), ((14, 78), 2), ((14, 84), 7), ((15, 52), 7), ((15, 105), 2), ((15, 56), 10), ((16, 106), 7), ((16, 76), 8), ((16, 111), 3), ((17, 22), 9), ((17, 40), 5), ((17, 68), 3), ((18, 79), 4), ((18, 105), 2), ((19, 50), 8), ((19, 122), 7), ((19, 35), 1), ((20, 119), 4), ((20, 128), 9), ((20, 29), 7), ((21, 73), 3), ((21, 58), 2), ((22, 64), 4), ((22, 135), 9), ((23, 48), 7), ((23, 50), 

In [5]:
import json

# Your exp_grid looks like:
# [{'N': 4, 'graph_id': 0, 'seed': 45854826, 'G': [((0, 3), 3), ...], 'depth': 1}, ...]

# STEP 1: Convert tuple keys to lists (JSON doesn’t support tuple)
def make_json_serializable(obj):
    if isinstance(obj, tuple):
        return list(obj)
    if isinstance(obj, list):
        return [make_json_serializable(x) for x in obj]
    if isinstance(obj, dict):
        return {k: make_json_serializable(v) for k, v in obj.items()}
    return obj

json_ready = [make_json_serializable(entry) for entry in exp_grid]

# STEP 2: Save to JSON file
with open("experiment_grid_high_N.json", "w") as f:
    json.dump(json_ready, f, indent=2)

print(f"Saved {len(json_ready)} entries to experiment_grid_high_N.json ✅")


Saved 30 entries to experiment_grid_high_N.json ✅


In [14]:
with open("experiment_grid.json", "r") as f:
    loaded = json.load(f)

# Convert lists back to tuples if you want
def convert_lists_to_tuples(obj):
    if isinstance(obj, list):
        # Check if list looks like an edge [(0,1), w]
        if len(obj) == 2 and isinstance(obj[0], list) and isinstance(obj[1], (int, float)):
            return (tuple(obj[0]), obj[1])
        else:
            return [convert_lists_to_tuples(x) for x in obj]
    if isinstance(obj, dict):
        return {k: convert_lists_to_tuples(v) for k, v in obj.items()}
    return obj

exp_grid_loaded = [convert_lists_to_tuples(entry) for entry in loaded]

print(exp_grid_loaded[0])  # should match your original structure


{'N': 4, 'graph_id': 0, 'seed': 45854826, 'G': [((0, 3), 3), ((0, 2), 1), ((0, 1), 5), ((1, 2), 7), ((1, 3), 8), ((2, 3), 8)], 'depth': 1}


TSP Graph GENERATION

In [57]:
from qiskit_optimization.applications import Tsp
import networkx as nx
def gengraph_tsp(N, seed=None):
    n = int(np.sqrt(N))
    tsp = Tsp.create_random_instance(n, seed=seed)
    qp = tsp.to_quadratic_program()

    val = nx.to_dict_of_dicts(tsp.graph)
    return val

In [63]:
N_list_tsp = [4, 9, 16, 25, 36, 49, 64, 81, 100]  # your Ns
per_n = 10                                     # 10 graphs per N
depths = [1]                   # SAME depth set used for every N
graph_runs_tsp = create_graphs_per_n(
    Ns=N_list_tsp,
    per_n=10,
    max_retries=100,
    base_seed=42,          # change if you want a different reproducible suite
    gengraph_fn=gengraph_tsp,  # or omit if gengraph is global
)


exp_grid_tsp = build_experiment_grid(graph_runs_tsp, depths)

print(f"Total graphs: {len(graph_runs_tsp)}  (expected {len(N_list_tsp)*per_n})")
print(f"Total (graph, depth) pairs: {len(exp_grid_tsp)}  (expected {len(graph_runs_tsp)*len(depths)})")
# Optional: quick peek at the first few items
# for row in exp_grid:
#     print(f"N={row['N']:<3} gid={row['graph_id']:<2} depth={row['depth']:<2} seed={row['seed']}")

print(print(exp_grid_tsp[50]['G']))

print(Tsp(exp_grid_tsp[50]['G']).to_quadratic_program())

Total graphs: 90  (expected 90)
Total (graph, depth) pairs: 90  (expected 90)
{0: {1: {'weight': np.float64(38.0)}, 2: {'weight': np.float64(13.0)}, 3: {'weight': np.float64(24.0)}, 4: {'weight': np.float64(73.0)}, 5: {'weight': np.float64(47.0)}, 6: {'weight': np.float64(32.0)}}, 1: {0: {'weight': np.float64(38.0)}, 2: {'weight': np.float64(50.0)}, 3: {'weight': np.float64(28.0)}, 4: {'weight': np.float64(42.0)}, 5: {'weight': np.float64(57.0)}, 6: {'weight': np.float64(6.0)}}, 2: {0: {'weight': np.float64(13.0)}, 1: {'weight': np.float64(50.0)}, 3: {'weight': np.float64(32.0)}, 4: {'weight': np.float64(82.0)}, 5: {'weight': np.float64(53.0)}, 6: {'weight': np.float64(44.0)}}, 3: {0: {'weight': np.float64(24.0)}, 1: {'weight': np.float64(28.0)}, 2: {'weight': np.float64(32.0)}, 4: {'weight': np.float64(51.0)}, 5: {'weight': np.float64(65.0)}, 6: {'weight': np.float64(22.0)}}, 4: {0: {'weight': np.float64(73.0)}, 1: {'weight': np.float64(42.0)}, 2: {'weight': np.float64(82.0)}, 3: {'we

In [59]:
json_ready_tsp = [make_json_serializable(entry) for entry in exp_grid_tsp]

# STEP 2: Save to JSON file
with open("experiment_grid_tsp.json", "w") as f:
    json.dump(json_ready_tsp, f, indent=2)

print(f"Saved {len(exp_grid_tsp)} entries to experiment_grid_tsp.json ✅")


Saved 90 entries to experiment_grid_tsp.json ✅


In [ ]:
with open("experiment_grid_tsp.json", "r") as f:
    loaded = json.load(f)

def keys_to_int(d):
    """Recursively convert all string keys that look like integers to int."""
    if isinstance(d, dict):
        new_dict = {}
        for k, v in d.items():
            # Convert numeric string keys to int
            new_key = int(k) if isinstance(k, str) and k.isdigit() else k
            new_dict[new_key] = keys_to_int(v)
        return new_dict
    else:
        return d

exp_grid_loaded = [keys_to_int(entry) for entry in loaded]

#print(exp_grid_loaded[50]['G'])  # should match your original structure

loaded = Tsp(exp_grid_loaded[50]['G'])
print(loaded.to_quadratic_program())

minimize 38*x_0_0*x_1_1 + 38*x_0_0*x_1_6 + 13*x_0_0*x_2_1 + 13*x_0_0*x_2_6 + 24*x_0_0*x_3_1 + 24*x_0_0*x_3_6 + 73*x_0_0*x_4_1 + 73*x_0_0*x_4_6 + 47*x_0_0*x_5_1 + 47*x_0_0*x_5_6 + 32*x_0_0*x_6_1 + 32*x_0_0*x_6_6 + 38*x_0_1*x_1_0 + 38*x_0_1*x_1_2 + 13*x_0_1*x_2_0 + 13*x_0_1*x_2_2 + 24*x_0_1*x_3_0 + 24*x_0_1*x_3_2 + 73*x_0_1*x_4_0 + 73*x_0_1*x_4_2 + 47*x_0_1*x_5_0 + 47*x_0_1*x_5_2 + 32*x_0_1*x_6_0 + 32*x_0_1*x_6_2 + 38*x_0_2*x_1_1 + 38*x_0_2*x_1_3 + 13*x_0_2*x_2_1 + 13*x_0_2*x_2_3 + 24*x_0_2*x_3_1 + 24*x_0_2*x_3_3 + 73*x_0_2*x_4_1 + 73*x_0_2*x_4_3 + 47*x_0_2*x_5_1 + 47*x_0_2*x_5_3 + 32*x_0_2*x_6_1 + 32*x_0_2*x_6_3 + 38*x_0_3*x_1_2 + 38*x_0_3*x_1_4 + 13*x_0_3*x_2_2 + 13*x_0_3*x_2_4 + 24*x_0_3*x_3_2 + 24*x_0_3*x_3_4 + 73*x_0_3*x_4_2 + 73*x_0_3*x_4_4 + 47*x_0_3*x_5_2 + 47*x_0_3*x_5_4 + 32*x_0_3*x_6_2 + 32*x_0_3*x_6_4 + 38*x_0_4*x_1_3 + 38*x_0_4*x_1_5 + 13*x_0_4*x_2_3 + 13*x_0_4*x_2_5 + 24*x_0_4*x_3_3 + 24*x_0_4*x_3_5 + 73*x_0_4*x_4_3 + 73*x_0_4*x_4_5 + 47*x_0_4*x_5_3 + 47*x_0_4*x_5_5 + 32*x_

In [1]:
import torch
import numpy as np
import json
import os

num_params = 8999          # number of parameters in your ansatz
seeds = list(range(10))       # 10 seeds: 0..9
save_dir = "theta_inits"      # folder to save all
os.makedirs(save_dir, exist_ok=True)

for seed in seeds:
    g = torch.Generator(device="cpu").manual_seed(seed)
    # Uniform angle in [-π, π]
    theta_init = (2 * torch.pi) * torch.rand(num_params, generator=g) - torch.pi

    # Save tensor
    torch.save(theta_init, f"{save_dir}/theta_init_s{seed}_extra.pt")